Phase 1: Data Ingestion<b>

In [124]:
#import Libraries
import json
import pandas as pd
import numpy as np

Load JSON data <br> Import raw JSON files into the environment using appropriate libraries (e.g., json, pandas). 
This step makes the raw data accessible for analysis and processing.

In [125]:
with open("Downloads/demo_json.json", "r") as file:
    data = json.load(file)

Inspect schema and structure<br>
Examine the overall structure of the JSON data to understand available fields, data types, and relationships between objects. This helps determine how the data should be transformed.

In [126]:
# Check first record
data[0]  

{'order_id': 'ORD1001',
 'order_date': '2025-01-10',
 'customer': {'customer_id': 'CUST01',
  'name': 'Rahul Sharma',
  'city': 'Delhi'},
 'items': [{'product_id': 'P101',
   'product_name': 'Laptop',
   'category': 'Electronics',
   'price': 55000,
   'quantity': 1},
  {'product_id': 'P102',
   'product_name': 'Mouse',
   'category': 'Electronics',
   'price': 500,
   'quantity': 2}],
 'payment': {'method': 'Credit Card', 'status': 'Success'}}

In [127]:
# Check number of records
len(data)  

6

In [128]:
# Check all keys in first record
data[0].keys() 

dict_keys(['order_id', 'order_date', 'customer', 'items', 'payment'])

Identify Nested Fields<br>
Detect fields that contain nested objects or arrays (such as products within items). Identifying these is necessary for flattening the data later.

In [129]:
# Inspect nested fields in first record
print(data[0]["customer"])
print(data[0]["items"])
print(data[0]["payment"])

{'customer_id': 'CUST01', 'name': 'Rahul Sharma', 'city': 'Delhi'}
[{'product_id': 'P101', 'product_name': 'Laptop', 'category': 'Electronics', 'price': 55000, 'quantity': 1}, {'product_id': 'P102', 'product_name': 'Mouse', 'category': 'Electronics', 'price': 500, 'quantity': 2}]
{'method': 'Credit Card', 'status': 'Success'}


In [130]:
# Programmatically identify nested fields
nested_cols = [k for k, v in data[0].items() if isinstance(v, (dict, list))]
nested_cols

['customer', 'items', 'payment']

Phase 2: DataFrame Creation

Create DataFrame from JSON <br>
Transform the JSON data into Pandas DataFrames to enable tabular data manipulation and analysis.

In [131]:
df = pd.DataFrame(data)
df

,order_id,order_date,customer,items,payment
0,ORD1001,2025-01-10,"{'customer_id': 'CUST01', 'name': 'Rahul Sharm...","[{'product_id': 'P101', 'product_name': 'Lapto...","{'method': 'Credit Card', 'status': 'Success'}"
1,ORD1002,2025-01-11,"{'customer_id': 'CUST02', 'name': 'Sneha Verma...","[{'product_id': 'P103', 'product_name': 'Mobil...","{'method': 'UPI', 'status': 'Success'}"
2,ORD1003,2025-01-12,"{'customer_id': 'CUST03', 'name': 'Amit Patel'...","[{'product_id': 'P201', 'product_name': 'Dinin...","{'method': 'Cash', 'status': 'Failed'}"
3,ORD1004,2025-01-13,"{'customer_id': 'CUST04', 'name': 'Neha Singh'...","[{'product_id': 'P104', 'product_name': 'Headp...","{'method': 'Debit Card', 'status': 'Success'}"
4,ORD1005,2025-01-14,"{'customer_id': 'CUST05', 'name': 'Karan Mehta...","[{'product_id': 'P301', 'product_name': 'Offic...","{'method': 'Net Banking', 'status': 'Success'}"
5,ORD1006,2025-01-15,"{'customer_id': 'CUST06', 'name': 'Pooja Iyer'...","[{'product_id': 'P105', 'product_name': 'Smart...","{'method': 'UPI', 'status': 'Success'}"


Flatten nested fields and expand arrays so that one row represents one product per order<br>
Flatten nested JSON and explode arrays using pandas.json_normalize(), creating one row per product per order while preserving order, customer, and payment details.

In [133]:
df = pd.json_normalize(
    data, 
    record_path=['items'], 
    meta=['order_id', 'order_date', ['customer', 'customer_id'], 
          ['customer', 'name'], ['customer', 'city'], 
          ['payment', 'method'], ['payment', 'status']],
    record_prefix='product_'
)
df.head()

,product_product_id,product_product_name,product_category,product_price,product_quantity,order_id,order_date,customer.customer_id,customer.name,customer.city,payment.method,payment.status
0,P101,Laptop,Electronics,55000,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success
1,P102,Mouse,Electronics,500,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success
2,P103,Mobile Phone,Electronics,32000,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success
3,P201,Dining Table,Furniture,18000,1,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed
4,P202,Chair,Furniture,3500,4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed


Phase 3: Data Cleaning

Handle missing or invalid values<br>
Identify null, missing, or incorrect values and address them using appropriate strategies such as imputation, replacement, or removal.

In [102]:
df.isnull().sum()

product_product_id    0
Product Name          0
Product Category      0
Unit Price            0
Quantity Sold         0
Order ID              0
Order Date            0
Customer ID           0
Customer Name         0
Customer City         0
Payment Method        0
payment.status        0
Total Amount          0
dtype: int64

In [137]:
#handle missing values if any
df['product_quantity'] = pd.to_numeric(df['product_quantity']).fillna(0)
df['product_price'] = pd.to_numeric(df['product_price']).fillna(0.0)

Validate data types<br>
Ensure each column has the correct data type (e.g., numeric, date, string) to prevent errors during calculations and analysis.

In [138]:
#numeric values
df['product_quantity'] = pd.to_numeric(df['product_quantity'], errors='coerce').astype(int)
df['product_price'] = pd.to_numeric(df['product_price'], errors='coerce').astype(float)

In [139]:
#date data type
df['order_date'] = pd.to_datetime(df['order_date'])

In [142]:
#string data types
df['customer.name'] = df['customer.name'].astype(str)
df['customer.city'] = df['customer.city'].astype(str)
df['payment.method'] = df['payment.method'].astype(str)
df['payment.status'] = df['payment.status'].astype(str)

Standardize column names<br>
Rename columns to follow a consistent naming convention (e.g., lowercase, snake_case) to improve readability and maintainability.

In [143]:
df.columns

Index(['product_product_id', 'product_product_name', 'product_category',
       'product_price', 'product_quantity', 'order_id', 'order_date',
       'customer.customer_id', 'customer.name', 'customer.city',
       'payment.method', 'payment.status', 'Total Amount'],
      dtype='object')

In [144]:
df = df.rename(columns={
    'order_id':'Order ID',
    'order_date':'Order Date',
    'customer.customer_id':'Customer ID',
    'customer.name':'Customer Name',
    'customer.city':'Customer City',
    'product_product_name':'Product Name',
    'product_category':'Product Category',
    'product_quantity':'Quantity Sold',
    'product_price':'Unit Price',  
    'payment.method':'Payment Method',
    'payment.status':'Payment Status' 
})
df

,product_product_id,Product Name,Product Category,Unit Price,Quantity Sold,Order ID,Order Date,Customer ID,Customer Name,Customer City,Payment Method,Payment Status,Total Amount
0,P101,Laptop,Electronics,55000.0,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,55000
1,P102,Mouse,Electronics,500.0,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,1000
2,P103,Mobile Phone,Electronics,32000.0,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,32000
3,P201,Dining Table,Furniture,18000.0,1,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,18000
4,P202,Chair,Furniture,3500.0,4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,14000
5,P104,Headphones,Electronics,2500.0,2,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,5000
6,P301,Office Chair,Furniture,12000.0,1,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,12000
7,P105,Smart Watch,Electronics,9000.0,2,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,18000


Phase 4: NumPy-based Transformations


Perform revenue calculations using NumPy arrays<br>
Convert the unit price and quantity sold columns to NumPy arrays and perform element-wise multiplication using np.multiply to calculate each product’s total revenue, storing the result in a new Total Amount column. Using NumPy arrays makes this efficient because operations are vectorized, allowing fast computation over the entire array without explicit Python loops.

In [146]:
prices = np.array(df['Unit Price'])
quantities = np.array(df['Quantity Sold'])
df['Total Amount'] = np.multiply(prices, quantities)
df

,product_product_id,Product Name,Product Category,Unit Price,Quantity Sold,Order ID,Order Date,Customer ID,Customer Name,Customer City,Payment Method,Payment Status,Total Amount
0,P101,Laptop,Electronics,55000.0,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,55000.0
1,P102,Mouse,Electronics,500.0,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,1000.0
2,P103,Mobile Phone,Electronics,32000.0,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,32000.0
3,P201,Dining Table,Furniture,18000.0,1,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,18000.0
4,P202,Chair,Furniture,3500.0,4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,14000.0
5,P104,Headphones,Electronics,2500.0,2,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,5000.0
6,P301,Office Chair,Furniture,12000.0,1,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,12000.0
7,P105,Smart Watch,Electronics,9000.0,2,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,18000.0


Apply conditional logic using vectorized operations<br>
Use np.where to create High Value Order Flag, setting True for rows where Total Amount > 30000 and False otherwise. This leverages vectorized conditional evaluation, allowing efficient, row-wise flag assignment across the entire DataFrame.

In [147]:
df['High Value Order Flag'] = np.where(df['Total Amount'] > 30000, True, False)
df

,product_product_id,Product Name,Product Category,Unit Price,Quantity Sold,Order ID,Order Date,Customer ID,Customer Name,Customer City,Payment Method,Payment Status,Total Amount,High Value Order Flag
0,P101,Laptop,Electronics,55000.0,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,55000.0,True
1,P102,Mouse,Electronics,500.0,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,1000.0,False
2,P103,Mobile Phone,Electronics,32000.0,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,32000.0,True
3,P201,Dining Table,Furniture,18000.0,1,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,18000.0,False
4,P202,Chair,Furniture,3500.0,4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,14000.0,False
5,P104,Headphones,Electronics,2500.0,2,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,5000.0,False
6,P301,Office Chair,Furniture,12000.0,1,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,12000.0,False
7,P105,Smart Watch,Electronics,9000.0,2,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,18000.0,False


Use vectorized operations (np.where) to standardize Payment Status to Order Status (Success → Completed, else Cancelled)

In [149]:
df['Order Status'] = np.where(df['Payment Status'] == 'Success', 'Completed', 'Cancelled')
df

,product_product_id,Product Name,Product Category,Unit Price,Quantity Sold,Order ID,Order Date,Customer ID,Customer Name,Customer City,Payment Method,Payment Status,Total Amount,High Value Order Flag,Order Status
0,P101,Laptop,Electronics,55000.0,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,55000.0,True,Completed
1,P102,Mouse,Electronics,500.0,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,1000.0,False,Completed
2,P103,Mobile Phone,Electronics,32000.0,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,32000.0,True,Completed
3,P201,Dining Table,Furniture,18000.0,1,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,18000.0,False,Cancelled
4,P202,Chair,Furniture,3500.0,4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,14000.0,False,Cancelled
5,P104,Headphones,Electronics,2500.0,2,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,5000.0,False,Completed
6,P301,Office Chair,Furniture,12000.0,1,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,12000.0,False,Completed
7,P105,Smart Watch,Electronics,9000.0,2,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,18000.0,False,Completed


Boolean masking and performance-efficient processing<br>
Use Boolean masking to efficiently filter rows based on conditions (e.g., Order Status == 'Completed' or High Value Order Flag == True). Masks are vectorized, applying the condition to the entire column at once, which avoids slow Python loops and ensures fast, memory-efficient, and readable filtering.

In [150]:
#Boolean mask for completed orders
completed_mask = df['Order Status'] == 'Completed'

# Apply mask to get only completed orders
completed_orders = df[completed_mask]

In [151]:
# Boolean mask for high-value orders
high_value_mask = df['High Value Order Flag'] == True

# Apply mask
high_value_orders = df[high_value_mask]

In [152]:
print("Number of high-value orders:", high_value_orders.shape[0])
print("Total revenue from high-value orders:", high_value_orders['Total Amount'].sum())


Number of high-value orders: 2
Total revenue from high-value orders: 87000.0


Phase 5: Business Logic Application

Exclude failed payments<br>
Filter the DataFrame to include only successful orders (Order Status == 'Completed') using a Boolean mask and reset the index.

In [110]:
success_mask = df['Order Status'] == 'Completed'
df_successful = df[success_mask].reset_index(drop=True)
df_successful

,product_product_id,Product Name,Product Category,Unit Price,Quantity Sold,Order ID,Order Date,Customer ID,Customer Name,Customer City,Payment Method,payment.status,Total Amount,High Value Order Flag,Order Status
0,P101,Laptop,Electronics,55000,1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,55000,True,Completed
1,P102,Mouse,Electronics,500,2,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,1000,False,Completed
2,P103,Mobile Phone,Electronics,32000,1,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,32000,True,Completed
3,P104,Headphones,Electronics,2500,2,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,5000,False,Completed
4,P301,Office Chair,Furniture,12000,1,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,12000,False,Completed
5,P105,Smart Watch,Electronics,9000,2,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,18000,False,Completed


Calculate total sales amount<br>
Computes the total sales by summing all values in the 'Total Amount' column of the DataFrame

In [153]:
total_sales_amount = df['Total Amount'].sum()
print("Total Sales Amount:", total_sales_amount)

Total Sales Amount: 155000.0


Identify high-value orders<br>
Identifies and filters all high-value orders by applying a boolean mask on the 'High Value Order Flag' column


In [112]:
high_value_mask = df['High Value Order Flag'] == True
high_value_orders = df[high_value_mask].reset_index(drop=True)

Assign order status flags<br>
Assigns order status by conditionally labeling orders as 'Completed' or 'Cancelled' based on payment success using vectorized logic


In [155]:
df['Order Status'] = np.where(df['Payment Status'] == 'Success', 'Completed', 'Cancelled')

Phase 6: Business-Ready Dataset

In [156]:
final_columns = ['Order ID', 'Order Date', 'Customer ID', 'Customer Name', 
                 'Customer City','Product Name', 'Product Category', 'Quantity Sold', 
                 'Total Amount','Payment Method', 'Order Status', 'High Value Order Flag']

df_final = df[final_columns]
df_final

,Order ID,Order Date,Customer ID,Customer Name,Customer City,Product Name,Product Category,Quantity Sold,Total Amount,Payment Method,Order Status,High Value Order Flag
0,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Laptop,Electronics,1,55000.0,Credit Card,Completed,True
1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Mouse,Electronics,2,1000.0,Credit Card,Completed,False
2,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,Mobile Phone,Electronics,1,32000.0,UPI,Completed,True
3,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Dining Table,Furniture,1,18000.0,Cash,Cancelled,False
4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Chair,Furniture,4,14000.0,Cash,Cancelled,False
5,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Headphones,Electronics,2,5000.0,Debit Card,Completed,False
6,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Office Chair,Furniture,1,12000.0,Net Banking,Completed,False
7,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,Smart Watch,Electronics,2,18000.0,UPI,Completed,False


In [157]:
#Produce flat, clean, and structured dataset ready for analytics
df_final.to_csv("final_dataset.csv", index=False)

Answers to Business Questions 

1. Total revenue per city

In [169]:
revenue_city = df.groupby('Customer City')['Total Amount'].sum().reset_index()
print("Total revenue per city\n",revenue_city)

Total revenue per city
   Customer City  Total Amount
0     Bangalore        5000.0
1       Chennai       18000.0
2         Delhi       68000.0
3        Mumbai       32000.0
4          Pune       32000.0


2. Product category with highest revenue

In [167]:
#group by product category and find highest revenue among them
top_category = (df.groupby('Product Category')['Total Amount'].sum().sort_values(ascending=False).head(1))
print("Product category with highest revenue\n", top_category)

Product category with highest revenue
 Product Category
Electronics    111000.0
Name: Total Amount, dtype: float64


3. Count of high-value orders

In [165]:
# Boolean mask for high-value orders
high_value_orders = df[df['High Value Order Flag']]

# Count unique orders flagged as high-value
high_value_order_count = high_value_orders['Order ID'].unique().sum()
print("Count of High-Value Orders:", high_value_order_count)

Count of High-Value Orders: ORD1001ORD1002


4. Average order value

In [117]:
# Calculate total per order
order_totals = df.groupby('Order ID')['Total Amount'].sum()

# Average order value
average_order_value = order_totals.mean()
print("Average Order Value:", round(average_order_value, 2))


Average Order Value: 25833.33


5. Most preferred payment method

In [118]:
# Count orders by payment method
preferred_payment = df.groupby('Payment Method')['Order ID'].nunique().sort_values(ascending=False)

# Top payment method
top_payment_method = preferred_payment.index[0]
print("Most Preferred Payment Method:", top_payment_method)


Most Preferred Payment Method: UPI


6. Top customers by revenue

In [119]:
# Aggregate revenue per customer
customer_revenue = df.groupby(['Customer ID', 'Customer Name'])['Total Amount'].sum().reset_index()

# Sort descending to get top customers
top_customers = customer_revenue.sort_values(by='Total Amount', ascending=False).head(5)
print("Top 5 Customers by Revenue:")
print(top_customers)

Top 5 Customers by Revenue:
  Customer ID Customer Name  Total Amount
0      CUST01  Rahul Sharma         56000
1      CUST02   Sneha Verma         32000
2      CUST03    Amit Patel         32000
5      CUST06    Pooja Iyer         18000
4      CUST05   Karan Mehta         12000


7. Completed vs cancelled orders

In [120]:
# Count orders by status
order_status_counts = df.groupby('Order Status')['Order ID'].nunique().reset_index()
order_status_counts.rename(columns={'Order ID': 'Count of Orders'}, inplace=True)
print("Completed vs Cancelled Orders:")
print(order_status_counts)

Completed vs Cancelled Orders:
  Order Status  Count of Orders
0    Cancelled                1
1    Completed                5
